# Prerequisites

Run ElasticSearch in Docker
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

# Import

In [1]:
import json
from elasticsearch import Elasticsearch
from openai import OpenAI

# Implement

In [ ]:
es_client = Elasticsearch(''